### Load necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import neuralsens.partial_derivatives as ns
from sklearn.model_selection import train_test_split
import torch
torch.manual_seed(1)
%matplotlib qt

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

### Create synthetic dataset to check behavior of functions

In [ ]:
samples = 100000
n_columns = 8
sm = np.random.normal(size=(samples,n_columns))
df = pd.DataFrame(sm, columns=['X' + str(x) for x in range(1,n_columns+1)])

### Check behavior of Jacobian function

#### Create output Y as linear function of inputs with some non-linear relationship

In [ ]:
df['Y'] = - 0.8 * df.X1 + 0.5 * df.X2 ** 2 - df.X3 * df.X4 + 0.1 * np.random.normal(size=(samples,)) 

#### Train MLP model using the data.frame created

In [ ]:
## Create random 80/20 % split
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'Y'].to_numpy(), df['Y'], test_size = 0.2, random_state = 5)

In [ ]:
X_train_tch = torch.FloatTensor(X_train).requires_grad_(True).to(device)
X_test_tch = torch.FloatTensor(X_test).requires_grad_(True).to(device)
y_train_tch = torch.FloatTensor(y_train.to_numpy()).to(device)
y_test_tch = torch.FloatTensor(y_test.to_numpy()).to(device)

#### Define MLP model torch class

In [ ]:
class MLP(torch.nn.Sequential):
    def __init__(self, input_size:int, output_size:int = 1, hidden_size:list = [10]):
        # Store layers to initiate sequential neural network
        layers           = []
        first = True
        for idx, neurons in enumerate(hidden_size):
            if first:
                layers += [torch.nn.Linear(input_size, neurons)]
                first = False
            else:
                layers += [torch.nn.Linear(hidden_size[idx-1], neurons)]
            layers += [torch.nn.Sigmoid()]
        layers += [torch.nn.Linear(hidden_size[idx-1], output_size)]
        super(MLP, self).__init__(*layers)

In [ ]:
model = MLP(input_size=n_columns, output_size=1, hidden_size=[15,15])
model = model.to(device)

#### Model training

In [ ]:
# Define error loss and optimizer
criterion = torch.nn.MSELoss()
lr = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

In [ ]:
# Check model performance before training
model.eval()
y_pred = model(X_test_tch)
before_train = criterion(y_pred.squeeze().to(device), y_test_tch)
print('Test loss before training' , before_train.item()) 

In [ ]:
# Train model
model.train()
epoch = 0
loss = before_train
path=[]
while loss.item() > 0.05:
    optimizer.zero_grad() # Reset the gradient
    epoch += 1
    # Forward pass
    y_pred = model(X_train_tch)
    # Compute Loss
    loss = criterion(y_pred.squeeze().to(device), y_train_tch)
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

In [ ]:
# Check model performance after training
model.eval()
y_pred = model(X_test_tch)
before_train = criterion(y_pred.squeeze().to(device), y_test_tch)
print('Test loss after training' , before_train.item())  

#### Execute jacobian function and check sensitivity metrics

In [ ]:
# Obtain parameters to perform jacobian
X = pd.DataFrame(X_train, columns=df.columns[df.columns != 'Y'])
y = pd.DataFrame(y_train, columns=['Y'])
sens_end_layer = 'last'
sens_end_input = False
sens_origin_layer = 0
sens_origin_input = True

In [ ]:
wts_torch = []
bias_torch = []
for name, param in model.named_parameters():
    if "weight" in name:
        wts_torch.append(param.detach().T.to(device))
    if "bias" in name:
        bias_torch.append(param.detach().to(device))
actfunc_torch = ["identity", "logistic", "logistic", "identity"]

In [ ]:
jacobian = ns.jacobian_mlp(wts_torch, bias_torch, actfunc_torch, X, y, use_torch=True, dev=device.type)

In [ ]:
# Check sensitivity metrics
# For X1, mean should be around -0.8
# For X2, X3, X4, std shall be much greater than their mean
# For X5, mean and std shall be near 0
jacobian.summary()

In [ ]:
jacobian.info()

In [ ]:
jacobian.plot()